<a href="https://colab.research.google.com/github/damianiRiccardo90/BHP/blob/master/C1-Basic_Networking_Tools/TCP_Proxy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Building a TCP Proxy

There are several reasons to masturbate